## importing neccesary libraries 

In [5]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

## spliting the folder to have test and train folder

In [6]:
#pip install split-folders

In [7]:
import splitfolders

In [9]:
splitfolders.ratio('Data',output='output',seed=1337,ratio=(.7,0.1,0.2))

## preprocessing the images(argumentation)

In [10]:
train_gen=ImageDataGenerator(rescale=1./255,shear_range=0.2,horizontal_flip=True,zoom_range=0.2)
test_gen=ImageDataGenerator(rescale=1./255)


## connecting the imageGenerator to our train and test folder

In [11]:
train_set=train_gen.flow_from_directory('output/train', target_size=(150,150),batch_size=32,class_mode='binary')
test_set=test_gen.flow_from_directory('output/test', target_size=(150,150),batch_size=32,class_mode='binary')

Found 5286 images belonging to 2 classes.
Found 1513 images belonging to 2 classes.


## model building

In [15]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32,input_shape=(150,150,3),kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))
model.add(tf.keras.layers.Conv2D(filters=32,input_shape=(150,150,3),kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))
model.add(tf.keras.layers.Conv2D(filters=32,input_shape=(150,150,3),kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))


model.compile(optimizer='SGD',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(train_set,validation_data=test_set,epochs=5)

Epoch 1/5
166/166 [==============================] - 406s 2s/step - loss: 0.6172 - accuracy: 0.6612 - val_loss: 0.4465 - val_accuracy: 0.8447
Epoch 2/5
166/166 [==============================] - 406s 2s/step - loss: 0.3754 - accuracy: 0.8420 - val_loss: 0.2945 - val_accuracy: 0.8810
Epoch 3/5
166/166 [==============================] - 407s 2s/step - loss: 0.3168 - accuracy: 0.8733 - val_loss: 0.3239 - val_accuracy: 0.8665
Epoch 4/5
166/166 [==============================] - 408s 2s/step - loss: 0.2913 - accuracy: 0.8785 - val_loss: 0.2653 - val_accuracy: 0.8883
Epoch 5/5
166/166 [==============================] - 414s 2s/step - loss: 0.2839 - accuracy: 0.8812 - val_loss: 0.2470 - val_accuracy: 0.8956


## Prediction with single data

In [20]:
from keras.preprocessing import image
val_sets=image.load_img('output/val/withoutmask/without_mask_16.jpg',target_size=(150,150))

In [21]:
val_sets=image.img_to_array(val_sets)
val_sets=np.expand_dims(val_sets,axis=0)

In [22]:
y_predict=model.predict(val_sets)
y_predict

array([[1.]], dtype=float32)

In [23]:
if y_predict[0][0]==1:
    prediction='No mask'
else:
    prediction='mask'

In [24]:
prediction

'No mask'

# 

In [101]:
model.save('mask_model.h5')

In [12]:
import tensorflow as tf
model=tf.keras.models.load_model('mask_model.h5')